In [1]:
import os
import dotenv

from swap.utils import get_algod_client
from swap.operations import create_validator_app, delete_validator_app, get_pool_logicsig
from swap.account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

creator = Account(os.environ.get('CREATOR_PK'))

print(f"Creator address: {creator.get_address()}")

Creator address: FE6UTVPOXD7HCTEYTG27P7KEDZCQFX7ECJANPTI76CMJSKKDQRYFVB4NWM


In [3]:
app_id = create_validator_app(client, creator)
print(f"App ID: {app_id}")

Waiting for confirmation
Transaction JMB3Q6CA6MH4QVUC7YAJMFBK2T3EPPLUGEEEFAV7UUOBP3HLSVCA confirmed in round 1978.
App ID: 21


In [4]:
logic_sig = get_pool_logicsig(client, app_id, 1, 0)
print(logic_sig.address())

FQ3W3IA4LKDFXC6JQCDVADL32B64BHEGVXJ5GAFU6WZYTBGHVUILME3CGM


In [5]:
delete_validator_app(client, creator, app_id)

Waiting for confirmation
Transaction EXZIXVRQCFWIEDJR3CT54NKNYV4WE75QJXSQ4CH4GGCGVX4FL73Q confirmed in round 1980.
